In [1]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [192]:
def get_grid(model):
    """ Esta función nos permite obtener el estado de los diferentes agentes.
        *param* model : Modelo del que obtendrá la información. 
        *return* una matriz con la información del estado de cada uno de los agentes."""
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        for contenido in content:
            if isinstance(contenido, RobotAgent): #(content == None)
                grid[x][y] = 3
            else:
                grid[x][y] = contenido.state
    return grid

""" Este es el agente del cuarto y su piso, dínde sus atributos son su estado y su siguiente estado """
class RoomAgent(Agent):
    Dirty = 1
    Clean = 0
    
    def __init__(self, pos, model, state = Clean):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.state = state
        #self.next_state = None

class RobotAgent(Agent):
    """ Este agente representa una celda del modelo."""
    def __init__(self, unique_id, model):
        
        """ Crea un agente con ua dirección no definida"""
        super().__init__(unique_id, model)
        self.move_to = None #where to move to
        #self.moves = 0
    
    def step(self):
        """ Este método especifica que debe realizar el agente. En este caso, si la celda está sucia la limpiara
        , en otro caso solo se moverá"""
        
        neighbours = self.model.grid.get_neighbors(self.pos, moore=True, include_center=True)
        
        for neighbor in neighbours:
            if neighbor.pos == self.pos:                
                if neighbor.state == neighbor.Dirty: 
                    neighbor.state = neighbor.Clean
                    self.move_to = self.pos
                
                else:
                    neighborhood = self.model.grid.get_neighborhood(self.pos, moore = True, include_center = False)
                    move_to = self.random.choice(neighborhood)
                    self.move_to = move_to
                break
                
        self.model.grid.move_agent(self, self.move_to)
            
class RoomModel(Model):
    """ Define nuestro modelo, los agentes y su entorno."""
    def __init__(self, width, height, num_agents, per_dirty_cells):
        self.num_agents = num_agents
        self.per_dirty_cells = per_dirty_cells
        self.clean_cells = 1 - per_dirty_cells
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        
        amount_dirty = int((width * height) * per_dirty_cells)
        amount_empty_cells = list(self.grid.empties)
        """ Repartimos las celdas que estarán sucias en el grid."""
        for cells in range(amount_dirty):
            empty_cells = random.choice(amount_empty_cells)
            room = RoomAgent(empty_cells, self)
            room.state = room.Dirty
            self.grid.place_agent(room, empty_cells)
            self.schedule.add(room)
            amount_empty_cells.remove(empty_cells)
        

        amount_empty_cells = list(self.grid.empties)
        """ Una vez repartidas las celdas sucias, repartimos las celdas limpias.""" 
        for cells in amount_empty_cells:
            room = RoomAgent(cells, self)
            #room.state = room.Clean
            self.grid.place_agent(room, cells)
            self.schedule.add(room)
            
        """ Repartimos los robots, en 1,1."""
        for num in range(num_agents):
            robot = RobotAgent(num, self)
            self.grid.place_agent(robot, (1,1))
            self.schedule.add(robot)
            
        self.datacollector = DataCollector(model_reporters = {'Grid': get_grid})
        
    def step(self):
        """ Ejecuta un paso de la simulación."""
        self.datacollector.collect(self)
        self.schedule.step()
            
    def per_clean(self):
        clean_cells_all = 0
        for (content_cells, x, y) in self.grid.coord_iter():
            for content in content_cells:
                if isinstance(content, RoomAgent) and content.state == content.Clean:
                    clean_cells_all += 1
        
        self.clean_cells = (clean_cells_all / (self.grid.width * self.grid.height))*10

In [193]:
# INFORMACION DE LA ROOM:
WIDTH = 80
HEIGHT = 80
PER_DIRTY_CELLS = 0.6
NUM_AGENTS = 90

#Tiempo de ejecución
MAX_GENERATIONS = 2
moves = 0

start_time = time.time()
model = RoomModel(WIDTH, HEIGHT, NUM_AGENTS, PER_DIRTY_CELLS)

while((time.time() - start_time) < MAX_GENERATIONS):
    model.step()
    moves = moves + 1
real_time = str(datetime.timedelta(seconds = (time.time() - start_time)))
clean = model.clean_cells * 10

In [194]:
# Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.
all_grid = model.datacollector.get_model_vars_dataframe()

In [195]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greens')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [197]:
print("Tamaño del Room: ", WIDTH, "x", HEIGHT)
print("Cantidad de robots limíando: ", NUM_AGENTS)
print("Pocentaje de suciedad en el Room: ", PER_DIRTY_CELLS)
print("Tiempo necesario hasta que todas las celdas estén limpias: ", real_time, "segundos")
print('Porcentaje de celdas limpias después del termino de la simulación:', clean)
print('Número de movimientos realizados por todos los agentes:', moves)
anim

Tamaño del Room:  80 x 80
Cantidad de robots limíando:  90
Pocentaje de suciedad en el Room:  0.6
Tiempo necesario hasta que todas las celdas estén limpias:  0:00:02.003479 segundos
Porcentaje de celdas limpias después del termino de la simulación: 4.0
Número de movimientos realizados por todos los agentes: 140
